
I used four columns: high, low, close, and volumn, and 30 days historical data to predict closing price in next 5 days.

In [ ]:
import numpy as np
import MySQLdb
import time

cnx = MySQLdb.connect(user='root', host='127.0.0.1', db='dig')
cursor = cnx.cursor()

query = "SELECT symbol, high, low, close, vol  FROM data ORDER BY symbol, date"
cursor.execute(query)
dataset = []
x = 0
for (symbol, high, low, close, vol) in cursor:
    dataset.append([symbol, high, low, close, vol])
    x = x + 1
    if (x % 5000 == 0):
        print(".", end="")

    
import os
from six.moves import cPickle as pickle

pickle_file = 'dataset.pickle'

try:
    f = open(pickle_file, 'wb')
    save = {
        'data': dataset,
    }
    pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
    f.close()
except Exception as e:
    print('Unable to save data to', pickle_file, ':', e)
    raise

In [1]:
import numpy as np
import MySQLdb
import time
from klepto.archives import file_archive

history_used = 30
price_predict = 5

#np.random.seed(0)

def load_history(num_dataset):
    
    t = time.time()
    
    dataset = np.ndarray(shape=(num_dataset, history_used, 4), dtype=np.float32)
    labels = np.ndarray(shape=(num_dataset, price_predict, 4), dtype=np.float32)
    symbols = {}
    num_total = 0;
    
    cnx = MySQLdb.connect(user='root', host='127.0.0.1', db='dig')
    cursor = cnx.cursor()
    
    query = """SELECT high, low, close, vol FROM data WHERE symbol=%s ORDER BY date LIMIT %s, %s"""
    query_sum = ("""SELECT * FROM (SELECT symbol, COUNT(*) c FROM DATA GROUP BY symbol) T WHERE C > %d ORDER BY symbol"""
                 % (history_used + price_predict + 1))

    cursor.execute(query_sum)
    for (symbol, count) in cursor:
        symbols[symbol] = count - history_used - price_predict
        num_total = num_total + count - history_used - price_predict
        
    cursor.close()
    
    snapshot = file_archive('snapshot', {}, serialized=True, cached=False)
    snapshot.load()
    if (len(snapshot) == 0):
        temp_s = np.arange(num_total)
        np.random.shuffle(temp_s)
        selector = temp_s[:num_dataset]
        num_data = 0
    else:
        num_data = snapshot['num_data']
        selector = snapshot['selector']
        dataset = snapshot['dataset']
        labels = snapshot['labels']
    
    cursor = cnx.cursor()

    for i in selector:
        temp_sum = 0
        
        for s in symbols.keys():
            if (i < temp_sum + symbols[s]):
                symbol = s
                break
            temp_sum = temp_sum + symbols[s]
        
        cursor.execute(query, (symbol, int(i-temp_sum), int(history_used+price_predict+1)))
        [f_high, f_low, f_close, f_vol] = cursor.fetchone()
        t_data = np.ndarray(shape=(history_used + price_predict, 4), dtype=np.float32)
        
        x = 0
        for (d_high, d_low, d_close, d_vol) in cursor:
            t_data[x, :] = [d_high/f_high, d_low/f_low, d_close/f_close, d_vol/f_close]
            x = x + 1
        
        dataset[num_data, :, :] = t_data[:history_used, :]
        labels[num_data, :, :] = t_data[history_used:, :]

        num_data = num_data + 1
        if (num_data % 5000 == 0):
            print(".", end="")
            if (num_data * 100 / num_dataset % 10 == 0):
                print("%s%%" % (num_data * 100 / num_dataset), end="")
        
        snapshot['num_data'] = num_data
        snapshot['selector'] = selector[num_data:]
        snapshot['dataset'] = dataset
        snapshot['labels'] = labels
        

    cursor.close()
    cnx.close()
    
    snapshot.drop()
    
    return dataset, labels


train_size = 200000
validation_size = 10000
test_size = 10000

print("Loading train set [", end="")
[train_dataset, train_labels] = load_history(train_size)
print("]")
print("Loading validation set [", end="")
[valid_dataset, valid_labels] = load_history(validation_size)
print("]")
print("Loading test set [", end="")
[test_dataset, test_labels] = load_history(test_size)
print("]")

import os
from six.moves import cPickle as pickle

pickle_file = 'dataset.pickle'

try:
    f = open(pickle_file, 'wb')
    save = {
        'train_dataset': train_dataset,
        'train_labels': train_labels,
        'valid_dataset': valid_dataset,
        'valid_labels': valid_labels,
        'test_dataset': test_dataset,
        'test_labels': test_labels,
    }
    pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
    f.close()
except Exception as e:
    print('Unable to save data to', pickle_file, ':', e)
    raise


Loading train set [

KeyboardInterrupt: 